In [1]:
import sys
import os
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K

In [2]:
K.clear_session()

In [3]:
data_validacion = './Ndatos2/validacion/'
data_entrenamiento = './Ndatos2/entrenamiento/'

In [4]:
epocas=100
longitud, altura = 100, 100

batch_size = 45
pasos = 100
validation_steps = 25
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 2
lr = 0.0004

In [5]:
entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

In [6]:
entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

validacion_generador = test_datagen.flow_from_directory(
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical')

Found 4576 images belonging to 2 classes.
Found 1145 images belonging to 2 classes.


In [7]:
from tensorflow.python.keras.layers import Conv2D, BatchNormalization

In [8]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(longitud, altura, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax')) # 2 because we have cat and dog classes

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 98, 98, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 98, 98, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 47, 47, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 47, 47, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 64)        0

In [9]:
from tensorflow.python.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [10]:
earlystop = EarlyStopping(patience=10)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

callbacks = [earlystop, learning_rate_reduction]

In [11]:
model.fit(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps,
    callbacks=callbacks
    )

Epoch 1/100
100/100 [==============================] - 24s 209ms/step - loss: 1.0927 - accuracy: 0.6879 - val_loss: 1.5450 - val_accuracy: 0.5404
Epoch 2/100
100/100 [==============================] - 19s 191ms/step - loss: 0.5295 - accuracy: 0.8087 - val_loss: 0.5139 - val_accuracy: 0.7609
Epoch 3/100
100/100 [==============================] - 19s 194ms/step - loss: 0.3578 - accuracy: 0.8531 - val_loss: 0.6087 - val_accuracy: 0.7129
Epoch 4/100
100/100 [==============================] - 19s 190ms/step - loss: 0.3185 - accuracy: 0.8727 - val_loss: 0.4977 - val_accuracy: 0.7964
Epoch 5/100
100/100 [==============================] - 19s 190ms/step - loss: 0.2659 - accuracy: 0.8939 - val_loss: 0.5961 - val_accuracy: 0.8062
Epoch 6/100
100/100 [==============================] - 19s 189ms/step - loss: 0.2361 - accuracy: 0.9100 - val_loss: 0.5383 - val_accuracy: 0.8124
Epoch 7/100
100/100 [==============================] - 19s 189ms/step - loss: 0.2154 - accuracy: 0.9093 - val_loss: 0.7157 -

In [10]:
target_dir = './modeloCaballoGallina/'
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
model.save('./modeloCaballoGallina/modelo.h5')
model.save_weights('./modeloCaballoGallina/pesos.h5')